In [17]:
import pandas as pd

pls = ["scala", "rust", "python"]

forest = pd.concat([pd.read_csv(f"{pl}_tree_results.csv").assign(pl=lambda _: pl) for pl in pls])
loops = pd.concat([pd.read_csv(f"{pl}_loop_results.csv").assign(pl=lambda _: pl) for pl in pls])


results = pd.concat([forest, loops])
    

In [18]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 0 to 16
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   target          134 non-null    int64  
 1   type            134 non-null    object 
 2   execution_time  134 non-null    float64
 3   pl              134 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 5.2+ KB


In [19]:
with open("tests.txt") as f:
    numbers_str = f.read().split(",")
    numbers = [int(n) for n in numbers_str]

percentiles = numbers[:numbers.index(1)]
random_numbers = numbers[numbers.index(1):]

print(percentiles)
print(random_numbers)
print(numbers)

[840483, 840483, 572512, 825904, 666079, 779157]
[1, 722399, 573422, 693046, 328616, 382766, 51614, 264845, 293550, 256233, 109486]
[840483, 840483, 572512, 825904, 666079, 779157, 1, 722399, 573422, 693046, 328616, 382766, 51614, 264845, 293550, 256233, 109486]


In [20]:
results["groups"] = results["type"] + "_" + results["pl"]

In [21]:
import plotly.express as px

for numbers in [percentiles, random_numbers]:
    result_subset = results[results["target"].isin(numbers)].copy()
    result_subset["i"] = result_subset["target"].apply(lambda x: numbers.index(x))
    result_subset = result_subset.sort_values(by=("i"))
    fig = px.line(result_subset, x="target", y="execution_time", color='groups')
    fig.update_xaxes(type='category')
    fig.show()

In [22]:
groups = results["groups"].unique().tolist()

In [23]:
groups_data = []

for group in groups:
    groups_data.append({
        "group": group,
        "type": "min",
        "res": results.loc[results["groups"]==group, "execution_time"].min(),
    })
    groups_data.append({
        "group": group,
        "type": "max",
        "res": results.loc[results["groups"]==group, "execution_time"].max(),
    })
    groups_data.append({
        "group": group,
        "type": "mean",
        "res": results.loc[results["groups"]==group, "execution_time"].mean(),
    })
df = pd.DataFrame.from_dict(groups_data)
df.head()

,group,type,res
0,binary_tree_scala,min,2060.25000
1,binary_tree_scala,max,7542.85000
2,binary_tree_scala,mean,2993.34375
3,bst_array_rust,min,908.00000
4,bst_array_rust,max,1467.00000


In [24]:
fig = px.bar(df, x="group", y="res", color="type")
fig.show()